In [11]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

# LLMToolSelectorMiddleware

一种常见的压缩手段，在调用主模型之前，使用大语言模型（LLM）智能地选择相关工具。LLM 工具选择器适用于以下场景：
- 拥有大量工具（10 个以上）的智能体，其中大多数工具对每个查询并不相关；
- 通过过滤无关工具来减少 token 使用量；
- 提高模型的专注度和准确性。

该中间件利用结构化输出，让 LLM 判断当前查询最相关的工具是哪些。

## 构造参数

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `model` | str \| BaseChatModel \| None | None | 用于选择的模型。如果未提供，使用代理的主模型。可以是模型标识符字符串或 BaseChatModel 实例 |
| `system_prompt` | str | `DEFAULT_SYSTEM_PROMPT` | 选择模型的指令 |
| `max_tools` | int \| None | None | 要选择的最大工具数。如果模型选择了更多，仅使用前 `max_tools` 个。如果未指定，则没有限制 |
| `always_include` | list[str] \| None | None | 无论选择如何都始终包含的工具名称。这些不计入 `max_tools` 限制 |

## 核心方法

| 方法 | 说明 |
|------|------|
| `_prepare_selection_request` | 准备工具选择的输入 |
| `_process_selection_response` | 处理选择响应并返回过滤后的 ModelRequest |
| `wrap_model_call` /`awrap_model_call` | 在通过处理器调用模型之前，基于 LLM 选择过滤工具 |

## 工作流程

1. `_prepare_selection_request` 逻辑

```python
def _prepare_selection_request(self, request: ModelRequest) -> _SelectionRequest | None:
    # 1. 检查是否有可用工具
    if not request.tools or len(request.tools) == 0:
        return None

    # 2. 过滤出 BaseTool 实例（排除提供商特定的工具字典）
    base_tools = [tool for tool in request.tools if not isinstance(tool, dict)]

    # 3. 验证 always_include 工具是否存在
    if self.always_include:
        available_tool_names = {tool.name for tool in base_tools}
        missing_tools = [
            name for name in self.always_include if name not in available_tool_names
        ]
        if missing_tools:
            raise ValueError(f"Tools in always_include not found: {missing_tools}")

    # 4. 分离始终包含的工具和可供选择的工具
    available_tools = [tool for tool in base_tools if tool.name not in self.always_include]

    # 5. 如果没有可供选择的工具，返回 None
    if not available_tools:
        return None

    # 6. 准备系统消息
    system_message = self.system_prompt
    if self.max_tools is not None:
        system_message += (
            f"\nIMPORTANT: List the tool names in order of relevance, "
            f"with the most relevant first. "
            f"If you exceed the maximum number of tools, "
            f"only the first {self.max_tools} will be used."
        )

    # 7. 获取最后一条用户消息
    for message in reversed(request.messages):
        if isinstance(message, HumanMessage):
            last_user_message = message
            break

    # 8. 返回选择请求
    return _SelectionRequest(
        available_tools=available_tools,
        system_message=system_message,
        last_user_message=last_user_message,
        model=self.model or request.model,
        valid_tool_names=[tool.name for tool in available_tools],
    )
```

2. `_process_selection_response` 逻辑

```python
def _process_selection_response(
    self,
    response: dict,
    available_tools: list[BaseTool],
    valid_tool_names: list[str],
    request: ModelRequest,
) -> ModelRequest:
    # 1. 提取选择的工具名称
    selected_tool_names: list[str] = []
    invalid_tool_selections = []

    # 2. 验证选择的工具
    for tool_name in response["tools"]:
        if tool_name not in valid_tool_names:
            invalid_tool_selections.append(tool_name)
            continue

        # 3. 应用 max_tools 限制
        if tool_name not in selected_tool_names and (
            self.max_tools is None or len(selected_tool_names) < self.max_tools
        ):
            selected_tool_names.append(tool_name)

    # 4. 检查无效选择
    if invalid_tool_selections:
        raise ValueError(f"Model selected invalid tools: {invalid_tool_selections}")

    # 5. 过滤工具
    selected_tools: list[BaseTool] = [
        tool for tool in available_tools if tool.name in selected_tool_names
    ]

    # 6. 添加 always_include 工具
    always_included_tools: list[BaseTool] = [
        tool
        for tool in request.tools
        if not isinstance(tool, dict) and tool.name in self.always_include
    ]
    selected_tools.extend(always_included_tools)

    # 7. 保留提供商特定的工具字典
    provider_tools = [tool for tool in request.tools if isinstance(tool, dict)]

    # 8. 返回修改后的请求
    return request.override(tools=[*selected_tools, *provider_tools])
```

3. `wrap_model_call` 逻辑

```python
def wrap_model_call(
    self,
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelCallResult:
    # 1. 准备选择请求
    selection_request = self._prepare_selection_request(request)
    if selection_request is None:
        return handler(request)

    # 2. 创建动态响应模型
    type_adapter = _create_tool_selection_response(selection_request.available_tools)
    schema = type_adapter.json_schema()
    structured_model = selection_request.model.with_structured_output(schema)

    # 3. 调用选择模型
    response = structured_model.invoke(
        [
            {"role": "system", "content": selection_request.system_message},
            selection_request.last_user_message,
        ]
    )

    # 4. 验证响应
    if not isinstance(response, dict):
        raise AssertionError(f"Expected dict response, got {type(response)}")

    # 5. 处理选择响应
    modified_request = self._process_selection_response(
        response,
        selection_request.available_tools,
        selection_request.valid_tool_names,
        request,
    )

    # 6. 调用处理器
    return handler(modified_request)
```

## 示例

In [18]:
from langchain_deepseek import ChatDeepSeek
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolSelectorMiddleware, AgentMiddleware
from langchain.agents.middleware.types import ModelRequest, ModelResponse, ModelCallResult
from langchain_core.tools import tool
from typing import Callable
from langchain_core.messages import HumanMessage

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

@tool
def check_weather(city: str) -> str:
    """查询指定城市的天气情况
    
    Args:
        city: 城市名称
        
    Returns:
        天气信息
    """
    weather_data = {
        "北京": "晴天，温度 25°C",
        "上海": "多云，温度 22°C",
        "广州": "阴天，温度 28°C",
        "深圳": "晴天，温度 27°C",
    }
    return weather_data.get(city, f"抱歉，没有找到 {city} 的天气信息")

@tool
def query_database(table: str, condition: str = "") -> str:
    """查询数据库中的数据
    
    Args:
        table: 表名
        condition: 查询条件（可选）
        
    Returns:
        查询结果
    """
    database_data = {
        "users": "用户表包含 1000 条记录",
        "orders": "订单表包含 5000 条记录",
        "products": "产品表包含 200 条记录",
    }
    if condition:
        return f"从 {table} 表查询，条件: {condition}，结果: {database_data.get(table, '表不存在')}"
    return f"查询 {table} 表，结果: {database_data.get(table, '表不存在')}"

@tool
def search_knowledge_base(query: str) -> str:
    """在知识库中搜索相关信息
    
    Args:
        query: 搜索关键词或问题
        
    Returns:
        知识库中的相关信息
    """
    knowledge_base = {
        "公司": "我们是一家专注于 AI 技术的公司",
        "产品": "主要产品包括智能客服、数据分析平台等",
        "技术": "使用最新的深度学习和自然语言处理技术",
        "服务": "提供 7x24 小时技术支持服务",
    }
    for key, value in knowledge_base.items():
        if key in query:
            return f"知识库搜索结果: {value}"
    return f"知识库中未找到关于 '{query}' 的信息"


class DebugToolSelectorMiddleware(LLMToolSelectorMiddleware):
    """调试工具选择中间件"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._call_count = 0
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelCallResult:
        self._call_count += 1
        call_id = self._call_count
        print(f"\n🔧 [Debug {call_id}] 🚀 ====== LLMToolSelectorMiddleware 开始 ======")
        print(f"🔧 [Debug {call_id}] 1. 传入的工具: {[tool.name for tool in request.tools]}")
        
        selection_request = self._prepare_selection_request(request)
        if selection_request is not None:
            print(f"🔧 [Debug {call_id}] 2. 选择模型看到的工具: {[tool.name for tool in selection_request.available_tools]}")
        
        print(f"🔧 [Debug {call_id}] 3. 调用 super().wrap_model_call()...")
        result = super().wrap_model_call(request, handler)
        print(f"🔧 [Debug {call_id}] 4. super().wrap_model_call() 返回")
        
        print(f"🔧 [Debug {call_id}] 5. result 类型: {type(result)}")
        print(f"🔧 [Debug {call_id}] 6. result 内容: {result}")
        if hasattr(result, 'response'):
            print(f"🔧 [Debug {call_id}] 7. result.response 类型: {type(result.response)}")
            if hasattr(result.response, 'content'):
                print(f"🔧 [Debug {call_id}] 8. 选择模型的选择: {result.response.content}")
        
        print(f"🔧 [Debug {call_id}] 🚗 ====== LLMToolSelectorMiddleware 完成 ======\n")
        
        return result


class LogSelectedTools(AgentMiddleware):
    """记录选择的工具"""
    
    def __init__(self):
        super().__init__()
        self._call_count = 0
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelCallResult:
        self._call_count += 1
        call_id = self._call_count
        print(f"📝 [Log {call_id}] 🚀 ====== LogSelectedTools 开始 ======")
        print(f"📝 [Log {call_id}] 看到的工具: {[tool.name for tool in request.tools]}")
        print(f"📝 [Log {call_id}] 调用 handler...")
        
        result = handler(request)
        
        print(f"📝 [Log {call_id}] handler 返回")
        print(f"📝 [Log {call_id}] 🚗 ====== LogSelectedTools 完成 ======\n")
        
        return result


tool_selector = DebugToolSelectorMiddleware(
    model=model,
    max_tools=1
)

agent = create_agent(
    model=model,
    tools=[check_weather, query_database, search_knowledge_base],
    middleware=[tool_selector, LogSelectedTools()]
)

print("=" * 50)
print("测试 1: 查询天气")
print("=" * 50)
result = agent.invoke({
    "messages": [HumanMessage("北京今天天气怎么样？")]
})
print(f"\n💬 回复: {result['messages'][-1].content}")

测试 1: 查询天气

🔧 [Debug 1] 🚀 ====== LLMToolSelectorMiddleware 开始 ======
🔧 [Debug 1] 1. 传入的工具: ['check_weather', 'query_database', 'search_knowledge_base']
🔧 [Debug 1] 2. 选择模型看到的工具: ['check_weather', 'query_database', 'search_knowledge_base']
🔧 [Debug 1] 3. 调用 super().wrap_model_call()...
📝 [Log 1] 🚀 ====== LogSelectedTools 开始 ======
📝 [Log 1] 看到的工具: ['check_weather']
📝 [Log 1] 调用 handler...
📝 [Log 1] handler 返回
📝 [Log 1] 🚗 ====== LogSelectedTools 完成 ======

🔧 [Debug 1] 4. super().wrap_model_call() 返回
🔧 [Debug 1] 5. result 类型: <class 'langchain.agents.middleware.types.ModelResponse'>
🔧 [Debug 1] 6. result 内容: ModelResponse(result=[AIMessage(content='我来帮您查询北京今天的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 319, 'total_tokens': 371, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 63}, 'model_provider'

流程总结：配置顺序`[tool_selector, LogSelectedTools]`意味着 tool_selector 在外层， LogSelectedTools 在内层。 tool_selector先执行wrap_model_call，过滤工具后传给 LogSelectedTools.wrap_model_call，所以 LogSelectedTools看到的是过滤后的1个工具。

# RetryMiddleware

ToolRetryMiddleware/ModelRetryMiddleware合并到一起来介绍，他们都支持自动对失败的调用进行重试

| 参数 | 类型 | 默认值 | ModelRetryMiddleware | ToolRetryMiddleware | 说明 |
|------|------|--------|---------------------|---------------------|------|
| `max_retries` | `number` | `2` | ✅ | ✅ | 初始调用失败后的最大重试次数（默认共尝试3次，包括初始调用） |
| `tools` | `list[BaseTool \| str]` | `None` | ❌ | ✅ | 可选的工具列表或工具名称列表，用于指定应用重试逻辑的范围。若为`None`，则对所有工具生效 |
| `retry_on` | `tuple[type[Exception], ...] \| callable` | `(Exception,)` | ✅ | ✅ | 可为一个异常类型元组，表示在这些异常发生时进行重试；也可为一个可调用对象，接收异常实例并返回`True`表示应重试 |
| `on_failure` | `string \| callable` | `"return_message"` | ✅ | ✅ | 当所有重试均失败后的处理行为。选项包括：`'return_message'`、`'raise'`、自定义可调用对象 |
| `backoff_factor` | `number` | `2.0` | ✅ | ✅ | 指数退避的乘数因子。每次重试的等待时间为 `initial_delay * (backoff_factor ** retry_number)` 秒。设为`0.0`表示使用固定延迟。随着失败次数增加，逐步拉长等待时间，给外部系统更多恢复机会 |
| `initial_delay` | `number` | `1.0` | ✅ | ✅ | 首次重试前的初始延迟时间（单位：秒） |
| `max_delay` | `number` | `60.0` | ✅ | ✅ | 重试之间延迟的最大值（单位：秒），用于限制指数退避的增长上限 |
| `jitter` | `boolean` | `true` | ✅ | ✅ | 是否在延迟中加入随机抖动（±25%），以避免"惊群效应"（thundering herd） |

**ToolRetryMiddleware 比 ModelRetryMiddleware 多一个 `tools` 参数，用于指定只重试特定工具。**

- `tools=None`：重试所有工具（默认）
- `tools=["tool1", "tool2"]`：只重试指定名称的工具
- `tools=[search_tool, db_tool]`：只重试指定的工具实例

工具重试示例

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import ToolRetryMiddleware


agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, database_tool, api_tool],
    middleware=[
        ToolRetryMiddleware(
            max_retries=3,
            backoff_factor=2.0,
            initial_delay=1.0,
            max_delay=60.0,
            jitter=True,
            tools=["api_tool"],
            retry_on=(ConnectionError, TimeoutError),
            on_failure="continue",
        ),
    ],
)

模型重试示例

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import ModelRetryMiddleware


# 基础用法：使用默认设置（2次重试，指数退避）
agent = create_agent(
    model="gpt-4o",
    tools=[search_tool],
    middleware=[ModelRetryMiddleware()],
)

# 自定义异常过滤
class TimeoutError(Exception):
    """自定义超时异常。"""
    pass

class ConnectionError(Exception):
    """自定义连接异常。"""
    pass

# 仅针对特定异常进行重试
retry = ModelRetryMiddleware(
    max_retries=4, # 最多重试4次
    retry_on=(TimeoutError, ConnectionError), # 仅在发生这两种异常时重试
    backoff_factor=1.5, # 退避因子
)


# 自定义重试判断逻辑
def should_retry(error: Exception) -> bool:
    # 仅在发生限流错误时重试
    if isinstance(error, TimeoutError):
        return True
    # 或者检查特定的 HTTP 状态码
    if hasattr(error, "status_code"):
        return error.status_code in (429, 503) # 429: 限流, 503: 服务不可用
    return False

# 使用自定义过滤函数
retry_with_filter = ModelRetryMiddleware(
    max_retries=3,
    retry_on=should_retry,
)

# 失败后返回错误信息而不是抛出异常
retry_continue = ModelRetryMiddleware(
    max_retries=4,
    on_failure="continue",  # 发生失败时返回 AIMessage 错误信息，而不是抛出异常
)

# 自定义错误信息格式
def format_error(error: Exception) -> str:
    return f"模型调用失败: {error}。请稍后再试。"

# 使用自定义格式化函数
retry_with_formatter = ModelRetryMiddleware(
    max_retries=4,
    on_failure=format_error,
)

# 使用固定退避时间（无指数增长）
constant_backoff = ModelRetryMiddleware(
    max_retries=5,
    backoff_factor=0.0,  # 禁用指数增长
    initial_delay=2.0,  # 每次固定等待 2 秒
)

# 严格模式：失败时直接抛出异常
strict_retry = ModelRetryMiddleware(
    max_retries=2,
    on_failure="error",  # 发生失败时直接重新抛出异常
)

# LLMToolEmulator

使用大语言模型（LLM）模拟工具执行，用于测试目的，将实际的工具调用替换为 AI 生成的响应。LLM 工具模拟器适用于以下场景：
- 在不执行真实工具的情况下测试智能体的行为；
- 在外部工具不可用或调用成本较高时开发智能体；
- 在实现真实工具之前，对智能体工作流进行原型设计和验证。

底层是依靠wrap_tool_call来实现的。

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `tools` | `list[str \| BaseTool] \| None` | `None` | 要模拟的工具列表。`None` 表示模拟所有工具，空列表表示不模拟任何工具 |
| `model` | `str \| BaseChatModel \| None` | `'anthropic:claude-sonnet-4-5-20250929'` | 用于模拟的模型。可以是模型标识符字符串或 `BaseChatModel` 实例 |

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulator
from langchain.tools import tool


# 定义获取天气的工具
@tool
def get_weather(location: str) -> str:
    """获取指定地点的当前天气。"""
    return f"Weather in {location}"

# 定义发送邮件的工具
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """发送邮件。"""
    return "Email sent"


# 方式一：模拟所有工具（默认行为）
# 如果 LLM 无法调用工具，中间件会尝试用自然语言模拟工具的执行结果
agent = create_agent(
    model="gpt-4o",
    tools=[get_weather, send_email],
    middleware=[LLMToolEmulator()],
)

# 方式二：仅模拟指定的工具
# 只对 get_weather 工具进行模拟，其他工具保持原样
agent2 = create_agent(
    model="gpt-4o",
    tools=[get_weather, send_email],
    middleware=[LLMToolEmulator(tools=["get_weather"])],
)

# 方式四：使用自定义模型进行模拟
# 指定使用 claude 模型来模拟工具的行为
agent4 = create_agent(
    model="gpt-4o",
    tools=[get_weather, send_email],
    middleware=[LLMToolEmulator(model="claude-sonnet-4-5-20250929")],
)

# ContextEditingMiddleware

当达到token限制时，通过清除较早的工具调用输出来管理对话上下文，同时保留最近的结果。这有助于在包含大量工具调用的长对话中，使上下文窗口保持在可控范围内。

该中间件适用于以下场景：
- 包含大量工具调用的长对话，其上下文超出令牌限制
- 通过移除不再相关的旧工具输出，降低令牌使用成本
- 仅在上下文中保留最近的 N 个工具调用结果
其工作原理如下：
- 监控对话中的令牌数量
- 当达到设定阈值时，清除较旧的工具输出
- 保留最近的 N 个工具调用结果
- 可选择性地保留工具调用的参数以维持上下文信息

ContextEditingMiddleware 参数

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `edits` | `Iterable[ContextEdit] \| None` | `ClearToolUsesEdit()` | 编辑策略序列，用于管理上下文大小 |
| `token_count_method` | `Literal["approximate", "model"]` | `"approximate"` | Token 计数方法：`"approximate"`（快速，不太准确）或 `"model"`（使用模型计数，更准确但较慢） |

ClearToolUsesEdit 参数

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `trigger` | `int` | `100_000` | 触发编辑的 token 阈值 |
| `clear_at_least` | `int` | `0` | 编辑运行时至少回收的 token 数量 |
| `keep` | `int` | `3` | 必须保留的最新工具结果数量 |
| `clear_tool_inputs` | `bool` | `False` | 是否清除 AI 消息上的工具调用参数 |
| `exclude_tools` | `Sequence[str]` | `()` | 不清除的工具名称列表 |
| `placeholder` | `str` | `"[cleared]"` | 为清除的工具输出插入的占位符文本 |

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import ContextEditingMiddleware, ClearToolUsesEdit


agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, your_calculator_tool, database_tool],
    middleware=[
        ContextEditingMiddleware(
            edits=[
                ClearToolUsesEdit(
                    trigger=2000,
                    keep=3,
                    clear_tool_inputs=False,
                    exclude_tools=[],
                    placeholder="[cleared]",
                ),
            ],
        ),
    ],
)

# ShellToolMiddleware

为Agent提供一个持久的shell工具，允许Agent执行系统命令。

| 功能 | 说明 |
|------|------|
| **持久会话** | 提供一个长期存在的 shell 会话，命令执行状态会被保留 |
| **执行策略** | 支持多种执行策略（主机、Codex 沙箱、Docker） |
| **安全控制** | 支持超时、输出限制、环境变量配置 |
| **PII 过滤** | 可配置 PII（个人身份信息）检测和脱敏规则 |
| **会话管理** | 支持启动命令、关闭命令、重启功能 |


## 参数

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `workspace_root` | `str \| Path \| None` | `None` | Shell 会话的基础目录，省略则创建临时目录 |
| `startup_commands` | `tuple[str, ...] \| list[str] \| str \| None` | `None` | 会话启动后执行的命令序列 |
| `shutdown_commands` | `tuple[str, ...] \| list[str] \| str \| None` | `None` | 会话关闭前执行的命令 |
| `execution_policy` | `BaseExecutionPolicy \| None` | `HostExecutionPolicy()` | 执行策略，控制超时、输出限制和资源配置 |
| `redaction_rules` | `tuple[RedactionRule, ...] \| list[RedactionRule] \| None` | `None` | 脱敏规则，用于清理命令输出中的敏感信息 |
| `tool_description` | `str \| None` | 默认描述 | 注册的 shell 工具的描述覆盖 |
| `tool_name` | `str` | `"shell"` | 注册的 shell 工具名称 |
| `shell_command` | `Sequence[str] \| str \| None` | `None` | 用于启动持久会话的 shell 可执行文件或参数序列 |
| `env` | `Mapping[str, Any] \| None` | `None` | 提供给 shell 会话的环境变量 |

## 执行策略

| 策略 | 说明 |
|------|------|
| `HostExecutionPolicy` | 完整主机访问，适合受信任的环境（Agent 已在容器或 VM 中运行） |
| `CodexSandboxExecutionPolicy` | 复用 Codex CLI 沙箱，提供额外的系统调用/文件系统限制 |
| `DockerExecutionPolicy` | 为每次 Agent 运行启动独立的 Docker 容器，提供更强的隔离 |

## 工作流程

1. Agent 调用 shell 工具（例如：`shell(command="ls -la")`）

2. 检查是否需要重启会话
   - 如果 `restart=True`：重启会话并执行启动命令

3. 执行命令
   - `result = session.execute(command, timeout)`

4. 处理超时
   - 如果超时：返回超时错误消息

5. 应用脱敏规则
   - 清理输出中的敏感信息
   - 如果检测到 PII：返回阻塞错误消息

6. 处理输出截断
   - 如果超过行数或字节数限制：添加截断提示

7. 处理退出码
   - 如果退出码非 0：添加退出码信息

8. 返回格式化的工具消息



In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    ShellToolMiddleware,          # Shell 工具中间件
    HostExecutionPolicy,          # 主机执行策略
    DockerExecutionPolicy,        # Docker 执行策略
    RedactionRule,                
)


# 基础用法：在主机上执行命令
agent = create_agent(
    model="gpt-4o",
    tools=[search_tool],
    middleware=[
        ShellToolMiddleware(
            workspace_root="/workspace",       # 工作空间根目录
            execution_policy=HostExecutionPolicy(), # 执行策略：直接在宿主机运行
        ),
    ],
)

# Docker 隔离模式：包含容器启动时的初始化命令
agent_docker = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ShellToolMiddleware(
            workspace_root="/workspace",
            startup_commands=[
                "pip install requests",         # 容器启动时安装依赖
                "export PYTHONPATH=/workspace"  # 设置环境变量
            ],
            execution_policy=DockerExecutionPolicy(
                image="python:3.11-slim",       # 使用的基础镜像
                command_timeout=60.0,           # 命令超时时间（秒）
            ),
        ),
    ],
)

# 带输出脱敏（红action）功能
agent_redacted = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ShellToolMiddleware(
            workspace_root="/workspace",
            redaction_rules=[
                # 定义脱敏规则：类型为 api_key，匹配正则表达式
                RedactionRule(pii_type="api_key", detector=r"sk-[a-zA-Z0-9]{32}"),
            ],
        ),
    ],
)

# FilesystemFileSearchMiddleware

提供基于文件系统的 Glob 和 Grep 搜索工具。文件搜索中间件适用于以下场景：
- 代码探索与分析
- 按文件名模式查找文件
- 使用正则表达式搜索代码内容
- 需要发现文件的大型代码库

该中间件为智能体（agents）添加了两个搜索工具：
- Glob 工具 — 快速文件模式匹配：
  - 支持如 **/*.py、src/**/*.ts 等模式
  - 返回按修改时间排序的匹配文件路径
- Grep 工具 — 基于正则表达式的内容搜索：
  - 支持完整的正则表达式语法
  - 可通过 include 参数按文件模式进行筛选
  - 提供三种输出模式：files_with_matches（含匹配项的文件）、content（匹配内容）、count（匹配数量）

##  参数

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `root_path` | `str` | 必填 | 搜索的根目录 |
| `use_ripgrep` | `bool` | `True` | 是否使用 `ripgrep` 进行搜索，如果不可用则回退到 Python |
| `max_file_size_mb` | `int` | `10` | 搜索的最大文件大小（MB） |

## 提供的工具

| 工具 | 功能 |
|------|------|
| `glob` | 快速文件模式匹配，支持 glob 模式如 `**/*.js` 或 `src/**/*.ts` |
| `grep` | 快速内容搜索，使用 ripgrep 或 Python 回退，支持正则表达式 |

> ripgrep（简称 rg ）是一个命令行工具，用于快速搜索文件内容，需要进行安装，如果未安装，会回退到python来进行实现。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import FilesystemFileSearchMiddleware
from langchain.messages import HumanMessage


agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        FilesystemFileSearchMiddleware(
            root_path="/workspace",      # 搜索的根目录
            use_ripgrep=True,            # 是否使用 ripgrep 工具进行搜索（更快）
            max_file_size_mb=10,         # 单个文件最大大小限制（MB）
        ),
    ],
)

# 现在 Agent 可以使用 glob_search 和 grep_search 工具了
result = agent.invoke({
    "messages": [HumanMessage("查找所有包含 'async def' 的 Python 文件")]
})

# Agent 的执行逻辑通常是：
# 1. 先用 glob_search(pattern="**/*.py") 找到所有 Python 文件
# 2. 再用 grep_search(pattern="async def", include="*.py") 在这些文件中搜索异步函数定义